# PRE-RUN PREP:

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone git repo
!git clone https://github.com/ethanrasmussen/llm_radiology.git

# Enter correct directory
%cd llm_radiology/
%ls

In [ ]:
# Install project requirements
!pip install -q -r requirements.txt

In [3]:
from dl4h.z0_utils import runcmd
import os
from PIL import Image
import torch
from torchvision import transforms
import pandas as pd

# BUILD DATASET:

In [ ]:
# NOTE: Removed prior to repo push for security, replace with your own credentials if re-running this notebook
PHYSIONET_USER = ""
PHYSIONET_PASS = ""

In [ ]:
# Download IMAGE_FILENAMES for MIMIC-CXR-JPG
IMAGE_FILENAMES_PATH = "/content/drive/MyDrive/LLM_CXR/IMAGE_FILENAMES"
# runcmd(f"wget -r -N -c -np --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -P {IMAGE_FILENAMES_PATH} https://physionet.org/files/mimic-cxr-jpg/2.1.0/IMAGE_FILENAMES")
!wget -r -N -c -np --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -P /content/drive/MyDrive/LLM_CXR/IMAGE_FILENAMES https://physionet.org/files/mimic-cxr-jpg/2.1.0/IMAGE_FILENAMES


In [ ]:
# Download subset of MIMIC-CXR-JPG
MIMIC_CXR_SUBSET_SIZE = 15000
MIMIC_CXR_DATASET_PATH = "/content/drive/MyDrive/LLM_CXR/raw_dataset"
# runcmd(f"head -n {MIMIC_CXR_SUBSET_SIZE} {IMAGE_FILENAMES_PATH}/physionet.org/files/mimic-cxr-jpg/2.1.0/IMAGE_FILENAMES |wget -r -N -c -np -nH --cut-dirs=1 --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -i - -P {MIMIC_CXR_DATASET_PATH} --base=https://physionet.org/files/mimic-cxr-jpg/2.1.0/")
!head -n 15000 /content/drive/MyDrive/LLM_CXR/IMAGE_FILENAMES/physionet.org/files/mimic-cxr-jpg/2.1.0/IMAGE_FILENAMES |wget -r -N -c -np -nH --cut-dirs=1 --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -i - -P /content/drive/MyDrive/LLM_CXR/raw_dataset --base=https://physionet.org/files/mimic-cxr-jpg/2.1.0/


In [ ]:
# Download MIMIC-CXR reports CSV
MIMIC_CXR_REPORTS_PATH = "/content/drive/MyDrive/LLM_CXR/raw_reports"
# runcmd(f"wget -r -N -c -np --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -P {MIMIC_CXR_REPORTS_PATH} https://physionet.org/files/mimic-cxr/2.1.0/mimic-cxr-reports.zip")
!wget -r -N -c -np --user {PHYSIONET_USER} --password {PHYSIONET_PASS} -P /content/drive/MyDrive/LLM_CXR/raw_reports https://physionet.org/files/mimic-cxr/2.1.0/mimic-cxr-reports.zip

In [ ]:
# runcmd(f"unzip {MIMIC_CXR_REPORTS_PATH}/physionet.org/files/mimic-cxr/2.1.0/mimic-cxr-reports.zip -d {MIMIC_CXR_REPORTS_PATH}")
!unzip /content/drive/MyDrive/LLM_CXR/raw_reports/physionet.org/files/mimic-cxr/2.1.0/mimic-cxr-reports.zip -d /content/drive/MyDrive/LLM_CXR/raw_reports

### PARSE & REORGANIZE DATASET
*NOTE: This can also be executed by dl4h/z0_parse_dataset.py*

In [ ]:
import os
import re
import csv
import shutil
import argparse

In [ ]:
def parse_study_report(path):
    """
    Read the .txt file at `path` and extract the INDICATION and IMPRESSION sections.
    Returns (indication, impression) as stripped strings, or (None, None) if either is missing.
    """
    text = open(path, 'r', encoding='utf-8', errors='ignore').read()
    # match from "INDICATION:" up to the next all-caps header (e.g. COMPARISON:, NOTIFICATION:, etc.) or EOF
    ind_pat = re.compile(r'INDICATION:(.*?)(?=\n\s*[A-Z ]{2,}:|\Z)', re.S)
    imp_pat = re.compile(r'IMPRESSION:(.*?)(?=\n\s*[A-Z ]{2,}:|\Z)', re.S)

    ind_m = ind_pat.search(text)
    imp_m = imp_pat.search(text)
    if not ind_m or not imp_m:
        return None, None

    indication = ind_m.group(1).strip().replace('\n', ' ')
    impression = imp_m.group(1).strip().replace('\n', ' ')
    return indication, impression

In [ ]:
def main_data_parse(text_root, image_root, output_dir, csv_tag=""):
    # ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # paths for our two CSVs
    indications_csv = os.path.join(output_dir, f'{csv_tag}{"_" if len(csv_tag)>0 else ""}indications.csv')
    reports_csv     = os.path.join(output_dir, f'{csv_tag}{"_" if len(csv_tag)>0 else ""}ground_truth_reports.csv')

    # open both CSVs for writing
    with open(indications_csv, 'w', newline='', encoding='utf-8') as ind_f, \
         open(reports_csv,     'w', newline='', encoding='utf-8') as rep_f:

        ind_writer = csv.writer(ind_f)
        rep_writer = csv.writer(rep_f)

        # write headers
        ind_writer.writerow(['study_id', 'indication'])
        rep_writer.writerow(['study_id', 'report'])

        # traverse all .txt files under the text_root
        for root, _, files in os.walk(text_root):
            for fname in files:
                if not fname.lower().endswith('.txt'):
                    continue

                study_id = os.path.splitext(fname)[0]
                txt_path = os.path.join(root, fname)

                # parse out the two sections
                indication, impression = parse_study_report(txt_path)
                if not indication or not impression:
                    # skip if either section was missing
                    continue

                # compute the matching image folder:
                # e.g. if txt_path is TEXT/files/p12/p1201675/s56699142.txt
                # then rel = p12/p1201675/s56699142, so image_dir = image_root/p12/p1201675/s56699142
                rel = os.path.relpath(txt_path, text_root)
                rel_dir = os.path.splitext(rel)[0]  # strips .txt
                image_dir = os.path.join(image_root, rel_dir)

                if not os.path.isdir(image_dir):
                    print(f"[WARN] no image folder for study {study_id}: {image_dir}")
                    continue

                # print(f"\n[!] Image folder located for {study_id}: {image_dir}\n")

                # pick the first JPG in that folder
                jpgs = sorted(f for f in os.listdir(image_dir)
                              if f.lower().endswith(('.jpg', '.jpeg')))
                if not jpgs:
                    print(f"[WARN] no JPG in {image_dir} for study {study_id}")
                    continue

                src_jpg = os.path.join(image_dir, jpgs[0])
                dst_jpg = os.path.join(output_dir, f"{study_id}.jpg")
                try:
                    # copy jpg to destination
                    shutil.copy2(src_jpg, dst_jpg)
                    # write rows to each CSV
                    ind_writer.writerow([study_id, indication])
                    rep_writer.writerow([study_id, impression])
                except Exception as e:
                    print(f"[ERROR] copying {src_jpg} -> {dst_jpg}: {e}")

In [ ]:
PARSED_DATASET_PATH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET"
MIMIC_CXR_DATASET_PATH = "/content/drive/MyDrive/LLM_CXR/raw_dataset"
MIMIC_CXR_REPORTS_PATH = "/content/drive/MyDrive/LLM_CXR/raw_reports"

main_data_parse(
    text_root=f"{MIMIC_CXR_REPORTS_PATH}/files/p10",
    image_root=f"{MIMIC_CXR_DATASET_PATH}/mimic-cxr-jpg/2.1.0/files/p10",
    output_dir=PARSED_DATASET_PATH
)

In [ ]:
raw_jpg_list = os.listdir(f"{MIMIC_CXR_DATASET_PATH}/mimic-cxr-jpg/2.1.0/files/p10")
len(raw_jpg_list)

In [ ]:
i=0
for patient in raw_jpg_list:
  main_data_parse(
    text_root=f"{MIMIC_CXR_REPORTS_PATH}/files/p10/{patient}",
    image_root=f"{MIMIC_CXR_DATASET_PATH}/mimic-cxr-jpg/2.1.0/files/p10/{patient}",
    output_dir=PARSED_DATASET_PATH,
    csv_tag=str(patient)
  )
  i+=1

In [ ]:
import pandas as pd
import glob
import os

def combine_csv_files(directory, output_file, file_name_tag=""):
    """Combines all CSV files in a directory into a single CSV file.

    Args:
        directory: The directory containing the CSV files.
        output_file: The name of the output CSV file.
    """
    all_filenames = glob.glob(os.path.join(directory, f"*{file_name_tag}.csv"))
    all_df = []
    for f in all_filenames:
        df = pd.read_csv(f)
        all_df.append(df)
    merged_df = pd.concat(all_df, ignore_index=True)
    merged_df.to_csv(output_file, index=False)

In [ ]:
combine_csv_files(PARSED_DATASET_PATH, f"{PARSED_DATASET_PATH}/COMPILED_INDICATIONS.csv", file_name_tag="_indications")

In [ ]:
combine_csv_files(PARSED_DATASET_PATH, f"{PARSED_DATASET_PATH}/COMPILED_GROUND_TRUTH_REPORTS.csv", file_name_tag="_ground_truth_reports")

### CREATE IMAGE TENSOR (.pt)

In [5]:
PARSED_DATASET_PATH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET"
import os
from PIL import Image
import torch
from torchvision import transforms
import pandas as pd

In [ ]:
# Process raw JPG images into tensor file
df = pd.read_csv(f"{PARSED_DATASET_PATH}/COMPILED_INDICATIONS.csv").fillna("")
study_ids = df['study_id'].astype(str).tolist()

In [ ]:
# build exact transforms used by DenseChexpertModel at training time
prep = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.485, 0.485],
        std =[0.229, 0.229, 0.229],
    ),
])

In [ ]:
# load jpgs in order of CSV
imgs = []
for sid in study_ids:
    path = os.path.join(PARSED_DATASET_PATH, f"{sid}.jpg")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Image for study_id={sid} not found at {path}")
    img = Image.open(path).convert("RGB")
    imgs.append(prep(img))

In [ ]:
# stack into one big Tensor [N,3,320,320] and save
images = torch.stack(imgs)
torch.save(images, f"{PARSED_DATASET_PATH}/IMAGE_TENSOR.pt")
print(f"Saved {len(imgs)} preprocessed images -> {PARSED_DATASET_PATH}/IMAGE_TENSOR.pt")

# PREP PRE-EXISTING COMPARISON MODELS:

In [ ]:
!pip install --quiet huggingface_hub
!huggingface-cli login

In [ ]:
!pip install -q peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.8 MB/s eta 0:00:00


In [ ]:
LLM_MODELS_DIR = "/content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES"
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModelForImageTextToText
from peft import PeftModel
import torch
import os

In [ ]:
!pip install -q --upgrade transformers

In [ ]:
#### RADIOLOGY LLAMA 2 ####
# Load base & adapter:
base = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base, "allen-eric/radiology-llama2")
# Merge adapter weights into base
model = model.merge_and_unload()
# Save model files
SAVE_DIR = f"{LLM_MODELS_DIR}/radiology-llama2"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf").save_pretrained(SAVE_DIR)

In [ ]:
#### CLINICAL-GPT ####
# FROM: https://huggingface.co/medicalai/ClinicalGPT-base-zh
# PAPER: https://arxiv.org/abs/2306.09968
# Load model directly
model = AutoModelForCausalLM.from_pretrained(
    "medicalai/ClinicalGPT-base-zh",
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalGPT-base-zh")
# Save model files
SAVE_DIR = f"{LLM_MODELS_DIR}/clinical-gpt"
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# PERFORM LLM INFERENCE:

In [8]:
# Input items:
LLM_MODEL_DIR_PATH = "/content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES"
VISION_MODEL_DIR_PATH = "/content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint"
INDICATION_CSV_PATH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv"
IMAGE_TENSOR_PATH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt"

# Output locations:
PREDICTED_LABELS_PATH = "/content/drive/MyDrive/LLM_CXR/RESULTS/labels_PragmaticLlama.csv"
REPORTS_PATH = "/content/drive/MyDrive/LLM_CXR/RESULTS/reports_PragmaticLlama.csv"


In [ ]:
!mkdir /content/drive/MyDrive/LLM_CXR/RESULTS

In [ ]:
## Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/pragmatic-llama \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS/labels_PragmaticLlama.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS/reports_PragmaticLlama.csv


In [ ]:
## Radiology Llama 2
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/radiology-llama2 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS/labels_RadiologyLlama2_retrest.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS/reports_RadiologyLlama2_retrest.csv

In [ ]:
## ClinicalGPT
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path medicalai/ClinicalGPT-base-zh \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS/labels_ClinicalGPT.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS/reports_ClinicalGPT.csv

In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/finetuned-pragmatic-llama3 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS/labels_FinetunedPL3.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS/reports_FinetunedPL3.csv


# EVALUATE RESULTS:

In [10]:
GROUND_TRUTH_REPORTS_CSV_PATH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv"
GENERATED_REPORTS_CSV_PATH = "/content/drive/MyDrive/LLM_CXR/RESULTS/reports" # + `_{model}.csv`

EVAL_OUTPUTS_PATH = "/content/drive/MyDrive/LLM_CXR/RESULTS/evaluation" # + `_{model}.csv`

!pip install --quiet f1chexbert bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
## Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS/reports_PragmaticLlama.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS/evaluation_PragmaticLlama.csv

In [ ]:
## Radiology Llama 2
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS/reports_RadiologyLlama2.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS/evaluation_RadiologyLlama2.csv

In [ ]:
## ClinicalGPT
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS/reports_ClinicalGPT.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS/evaluation_ClinicalGPT.csv

In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS/reports_FinetunedPL3.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS/evaluation_FinetunedPL3.csv

# Chain-of-Thought Trial:

In [ ]:
!mkdir /content/drive/MyDrive/LLM_CXR/RESULTS_CoT

In [ ]:
## Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/pragmatic-llama \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/labels_PragmaticLlama.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_PragmaticLlama.csv \
    --instruct_path ./prompts/report_writing/CoT_instructions.txt


In [ ]:
## Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_PragmaticLlama.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/evaluation_PragmaticLlama.csv

In [ ]:
## Radiology Llama 2
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/radiology-llama2 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/labels_RadiologyLlama2.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_RadiologyLlama2.csv \
    --instruct_path ./prompts/report_writing/CoT_instructions.txt


In [ ]:
## Radiology Llama 2
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_RadiologyLlama2.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/evaluation_RadiologyLlama2.csv

In [ ]:
## ClinicalGPT
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path medicalai/ClinicalGPT-base-zh \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/labels_ClinicalGPT.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_ClinicalGPT.csv \
    --instruct_path ./prompts/report_writing/CoT_instructions.txt

In [ ]:
## ClinicalGPT
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_ClinicalGPT.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/evaluation_ClinicalGPT.csv

In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/finetuned-pragmatic-llama3 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/labels_FinetunedPL3.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_FinetunedPL3.csv \
    --instruct_path ./prompts/report_writing/CoT_instructions.txt


In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_FinetunedPL3.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_CoT/evaluation_FinetunedPL3.csv

# Structured Trial:

In [ ]:
!mkdir /content/drive/MyDrive/LLM_CXR/RESULTS_Structured

In [ ]:
## Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/pragmatic-llama \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/labels_PragmaticLlama.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_PragmaticLlama.csv \
    --instruct_path ./prompts/report_writing/structured_instructions.txt


In [ ]:
## Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_PragmaticLlama.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/evaluation_PragmaticLlama.csv

In [ ]:
## Radiology Llama 2
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/radiology-llama2 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/labels_RadiologyLlama2.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_RadiologyLlama2.csv \
    --instruct_path ./prompts/report_writing/structured_instructions.txt


In [ ]:
## Radiology Llama 2
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_RadiologyLlama2.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/evaluation_RadiologyLlama2.csv

In [ ]:
## ClinicalGPT
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path medicalai/ClinicalGPT-base-zh \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/labels_ClinicalGPT.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_ClinicalGPT.csv \
    --instruct_path ./prompts/report_writing/structured_instructions.txt

In [ ]:
## ClinicalGPT
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_ClinicalGPT.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/evaluation_ClinicalGPT.csv

In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python pragmatic_llama_inference.py --llama-path {LLM_MODEL_DIR_PATH} --vision_path {VISION_MODEL_DIR_PATH} --indication_path {INDICATION_CSV_PATH} --image_path {IMAGE_TENSOR_PATH} --vision_out_path {PREDICTED_LABELS_PATH} --outpath {REPORTS_PATH}")

!python pragmatic_llama_inference.py \
    --llama_path /content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/finetuned-pragmatic-llama3 \
    --vision_path /content/drive/MyDrive/LLM_RADIOLOGY/vision_resnet_checkpoint \
    --indication_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_INDICATIONS.csv \
    --image_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/IMAGE_TENSOR.pt \
    --vision_out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/labels_FinetunedPL3.csv \
    --outpath /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_FinetunedPL3.csv \
    --instruct_path ./prompts/report_writing/structured_instructions.txt


In [ ]:
## FinetunedPL3: Pragmatic Llama
# runcmd(f"python evaluate.py --gt_path {GROUND_TRUTH_REPORTS_CSV_PATH} --gen_path {GENERATED_REPORTS_CSV_PATH} --out_path {EVAL_OUTPUTS_PATH}")

!python evaluate.py --gt_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv \
    --gen_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_FinetunedPL3.csv \
    --out_path /content/drive/MyDrive/LLM_CXR/RESULTS_Structured/evaluation_FinetunedPL3.csv

# Impression Pruning Approach:

In [ ]:
import os
from pathlib import Path
import pandas as pd
from transformers import pipeline

In [12]:
models = ["FinetunedPL3", "ClinicalGPT", "RadiologyLlama2", "PragmaticLlama"]
prompting_approaches = ["_Structured", "_CoT", ""]

unpruned_results = []
for pa in prompting_approaches:
    unpruned_results += [f"/content/drive/MyDrive/LLM_CXR/RESULTS{pa}/reports_{model}.csv" for model in models]

In [13]:
unpruned_results

['/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/reports_PragmaticLlama.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/reports_PragmaticLlama.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/reports_PragmaticLlama.csv']

In [ ]:
MODEL_ID = "google/flan-t5-xl"

pipe = pipeline(
  task="text2text-generation",
  model=MODEL_ID,
  trust_remote_code=True,
  device_map="auto",
)

In [15]:
PRUNING_INSTRUCTIONS = """You are a researcher building a large language model for generating radiology reports. Some of your models output additional formatting information which is irrelevant. Your task is to take un 'unclean' report, which contains this additional formatting information, and isolate only the 'clean' radiology report text. Sometimes, the report you are given may not need any cleaning, and can be returned as-is. Other times, it will need pruning. Please see more details below.\n
Cleaned reports, sometimes called 'impressions', look like the following:\n
 - No acute cardiopulmonary process. \n
 - Mild pulmonary vascular congestion. Bibasilar opacities are likely due to atelectasis. Cardiomegaly. \n
 - No relevant change. No pneumonia, no pulmonary edema, no pleural effusions. Normal size of the heart. \n
 \n
 In contrast, unclean reports may contain added info such as justifications, history, chains of thought, etc. Please review the following unclean reports, as well as how they look after cleaning/pruning.\n
 Unclean Report 1:\n
 No pulmonary nodules or mass.  No pneumonia.  No pleural effusions.  No pulmonary edema.  No pneumothorax.  Normal mediastinal contours.  Normal cardiac silhouette.  Normal hilar and mediastinal contours.  No acute osseous abnormalities.\n
 ### Output:\n
 No acute cardiopulmonary abnormalities.  No evidence of pulmon\n
 Cleaned Report 1 (only the impression):\n
 No acute cardiopulmonary abnormalities.  No evidence of pulmon\n
 \n
 Uncleaned Report 2:\n
 Clinical Reasoning:\n
 -\n
 Final Impression:\n
 -\n
 - No evidence of pneumonia.\n
 Cleaned Report 2 (only the impression):\n
 No evidence of pneumonia.\n
 Uncleaned Report 3:\n
 1. The patient has a cardiomegaly, which can be associated with heart failure, leading to pulmonary edema.\n
 2. There are areas of increased opacity, suggesting fluid accumulation in the lungs.\n
 3. These findings are consistent with pulmonary edema.\n
 ### Impression:\n
 Pulmonary edema.\n
 Cleaned Report 3 (only the impression):\n
 Pulmonary edema.\n
 \n
 You must not change or rephrase the intended impression/report. Instead, eliminate unnecessary information that is added in addition to it.\n
 If the report is totally fine, simply repeat the report without ANY additional phrases. Now, let's begin."""

In [ ]:
# Iterate through all reports CSV files
for result in unpruned_results:
    df = pd.read_csv(result)
    pruned_reports = []
    # Iterate through all rows
    for idx, row in df.iterrows():
        # Apply prune instructions to each report
        report = str(row["report"])
        pruned_report = pipe(
            f"{PRUNING_INSTRUCTIONS}\nPLEASE CLEAN THIS FOLLOWING REPORT:\n{report}",
            max_new_tokens=256,
            do_sample=False,
            # truncation=True
        )[0]["generated_text"]
        # print(f"\nPRUNED:\n{pruned_report}\n")
        pruned_reports.append(pruned_report)
        print(f"Pruned {idx+1}/{len(df)} reports from {result}...")
    # Replace original reports with pruned
    df["report"] = pruned_reports
    # Save under new CSV filename
    df.to_csv(result.replace("reports", "pruned_reports"), index=False)
    print(f"Completed pruning on {result}\n\n")

In [17]:
pruned_results = [result.replace("reports", "pruned_reports") for result in unpruned_results]
GROUND_TRUTH = "/content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv"

pruned_results

['/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/pruned_reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/pruned_reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/pruned_reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_Structured/pruned_reports_PragmaticLlama.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/pruned_reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/pruned_reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/pruned_reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS_CoT/pruned_reports_PragmaticLlama.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/pruned_reports_FinetunedPL3.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/pruned_reports_ClinicalGPT.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/pruned_reports_RadiologyLlama2.csv',
 '/content/drive/MyDrive/LLM_CXR/RESULTS/pruned_reports_PragmaticLlama.csv']

In [ ]:
# Despite instructions, 'No relevant change.' or 'Output:' appears in many items, this removes it:
REMOVE_TEXT = ["No relevant change. ", "No relevant change.", "No relevant change", "No significant change. ", "No significant change.", "No significant change", "Output: ", "Output:", "Output", "Impression: ", "Impression:"]
for results_csv in pruned_results:
    df = pd.read_csv(results_csv)
    pruned_reports = []
    # Iterate through all rows
    print(f"Cleaning up {results_csv}...")
    for idx, row in df.iterrows():
        # Remove unnecessary 'No relevant change.' text
        report = str(row["report"])
        for remove in REMOVE_TEXT:
            report = report.replace(remove, "")
        pruned_reports.append(report)
    # Replace original reports with pruned
    df["report"] = pruned_reports
    # Updated file saved under CSV filename
    df.to_csv(results_csv, index=False)
    print(f"Cleaned up! -- {result}\n\n")

In [ ]:
for results_csv in pruned_results:
    print(f"Evaluating {results_csv}...")
    EVAL_PATH = results_csv.replace("pruned_reports", "pruned_evaluation")
    !python evaluate.py --gt_path {GROUND_TRUTH} --gen_path {results_csv} --out_path {EVAL_PATH}
    print(f"Evaluated {results_csv}!\n")